In [258]:
import yfinance as yf

%matplotlib inline

In [259]:
# start = '2019-01-01'
# end = '2021-02-03'

In [260]:
data = pd.read_csv('NiftyHistoricalData.csv')
# data = pd.read_csv('NiftyHistoricalData.csv', index_col=0)
# data = yf.download('GME', start = start, end = end)

In [261]:
display(data)
display(px.line(data, x='Date', y='Close'))

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0
1,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0
2,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0
3,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0
4,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0
...,...,...,...,...,...,...,...
3765,2023-01-27,17877.199219,17884.750000,17493.550781,17604.349609,17604.349609,476300
3766,2023-01-30,17541.949219,17709.150391,17405.550781,17648.949219,17648.949219,432400
3767,2023-01-31,17731.449219,17735.699219,17537.550781,17662.150391,17662.150391,398300
3768,2023-02-01,17811.599609,17972.199219,17353.400391,17616.300781,17616.300781,512900


In [262]:
import plotly.express as px
data['all-time-high'] = data['Close'].expanding().max()
data['pullback'] = 1 - data['Close'] / data['all-time-high']


display(px.line(data, x='Date', y=['Close', 'all-time-high']))
display(px.line(data, x='Date', y='pullback'))

In [263]:
def get_signal(data):
    pullback = data['pullback']
    
    if pullback >= 0.4:
        return '>40% dd'
    elif pullback >= 0.2:
        return '>20% dd'
    elif pullback >= 0.1:
        return '>10% dd'
    else: 
        return '< 10% dd'

data['signal'] = data.apply(get_signal, axis=1)
print (data)
display(px.scatter(data, x='Date', y='pullback', color='signal'))

data['count_value'] = 1
display(px.pie(data.groupby('signal').agg({'count_value': 'count'}).reset_index(), values='count_value', names='signal'))

            Date          Open          High           Low         Close  \
0     2007-09-17   4518.450195   4549.049805   4482.850098   4494.649902   
1     2007-09-18   4494.100098   4551.799805   4481.549805   4546.200195   
2     2007-09-19   4550.250000   4739.000000   4550.250000   4732.350098   
3     2007-09-20   4734.850098   4760.850098   4721.149902   4747.549805   
4     2007-09-21   4752.950195   4855.700195   4733.700195   4837.549805   
...          ...           ...           ...           ...           ...   
3765  2023-01-27  17877.199219  17884.750000  17493.550781  17604.349609   
3766  2023-01-30  17541.949219  17709.150391  17405.550781  17648.949219   
3767  2023-01-31  17731.449219  17735.699219  17537.550781  17662.150391   
3768  2023-02-01  17811.599609  17972.199219  17353.400391  17616.300781   
3769  2023-02-02  17517.099609  17653.900391  17445.949219  17610.400391   

         Adj Close  Volume  all-time-high  pullback    signal  
0      4494.649902     

In [269]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp, comment):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        self.comment = comment
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, data, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = data
        self.trading_allowed = True
        
    def get_positions_data(self):
        data = pd.DataFrame([position._asdict() for position in self.positions])
        data['pnl'] =data['profit'].cumsum() + self.starting_balance
        return data
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
    
    def trade(self, pullback, data):
        self.trading_allowed = True
        if data.signal == pullback:
            for pos in self.positions:
                if pos.status == 'open' and pos.comment == pullback:
                    self.trading_allowed = False
                    break
            if self.trading_allowed:
#                 self.add_position(Position(data.time, data.close, 'buy', self.volume, 0.0, 0.0, pullback))
                self.add_position(Position(data.Date, data.Close, 'buy', self.volume, 0.0, 0.0, pullback))
        
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            # opening positions at different dd levels
            self.trade('>10% dd', data)
            self.trade('>20% dd', data)
            self.trade('>40% dd', data)
            
            # if pullback is 0, close all positions
            if data.pullback == 0.0:
                for pos in self.positions:
                    if pos.status == 'open':
#                         pos.close_position(data.time, data.close)
                         pos.close_position(data.Date, data.Close)
        
        return self.get_positions_data()

In [270]:
buy_dd_strategy = Strategy(data, 10000, 1)
result = buy_dd_strategy.run()

display(result)
display(px.line(result, x='open_datetime', y='pnl'))

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2008-01-21,5208.799805,buy,1,0.0,0.0,2010-11-09,6301.549805,1092.750000,closed,11092.750000
1,2008-01-22,4899.299805,buy,1,0.0,0.0,2010-11-09,6301.549805,1402.250000,closed,12495.000000
2,2008-10-06,3602.350098,buy,1,0.0,0.0,2010-11-09,6301.549805,2699.199707,closed,15194.199707
3,2011-01-14,5654.549805,buy,1,0.0,0.0,2013-11-01,6307.200195,652.650391,closed,15846.850098
4,2011-08-16,5035.799805,buy,1,0.0,0.0,2013-11-01,6307.200195,1271.400391,closed,17118.250488
5,2015-05-07,8057.299805,buy,1,0.0,0.0,2017-03-14,9087.000000,1029.700195,closed,18147.950684
6,2016-02-11,6976.350098,buy,1,0.0,0.0,2017-03-14,9087.000000,2110.649902,closed,20258.600586
7,2018-03-23,9998.049805,buy,1,0.0,0.0,2018-07-24,11134.299805,1136.250000,closed,21394.850586
8,2018-10-05,10316.450195,buy,1,0.0,0.0,2019-04-16,11787.150391,1470.700195,closed,22865.550781
9,2019-08-05,10862.599609,buy,1,0.0,0.0,2019-11-27,12100.700195,1238.100586,closed,24103.651367


In [271]:
#to compare buy and hold vs do nothing approach
# fig = px.line(data, x='Date', y=['Close', 'all-time-high'], title='Buy & hold Strategy')

# # adding trades to plots
# for i, position in result.iterrows():
#     if position.status == 'closed':
#         fig.add_shape(type="line",
#             x0=position.open_datetime, y0=position.open_price, x1=position.close_datetime, y1=position.close_price,
#             line=dict(
#                 color="green" if position.profit >= 0 else "red",
#                 width=3)
#             )
# fig

In [ ]:
#calculate z-score 
std = data['Close'].rolling(20).std()
def zscore (data):
    return (data[-1] - data.mean())/data.std()

rolled = data.Close.rolling (window = 30).apply(zscore)
rolled.plot()

#calculate histgram for the rolled zscore
rolled.hist(bins=20)

#calculate the change for the percentage difference of the closing price 
min_pct_change = (data.Close.pct_change().rolling (window=30).min())
min_pct_change.plot()

#plot the histgram for the percentage change 
min_pct_change.hist(bins = 20)

In [ ]:
### Dip buying stategy##
def dip_buying_strategy(data, window=20, threshold=0.03):
    # Calculate the rolling average and standard deviation of the stock price
    avg = data['Close'].rolling(window).mean()
    std = data['Close'].rolling(window).std()

    # Calculate the upper and lower bounds for the "dip"
    upper_bound = avg + std * threshold
    lower_bound = avg - std * threshold
    
    # Create a new column to indicate whether to buy or not
    data['Signal'] = np.where(data['Close'] < lower_bound, 1, 0)

    
    return data

In [ ]:
dip_buying_strategy (data,20,0.03)

In [ ]:
def calculate_z_score(price_data, window=100):
    """
    Calculate Z-score of a stock's price data.
    
    Parameters:
        price_data (pandas Series): Stock's price data.
        window (int, optional): The number of data points to use for calculation.
    
    Returns:
        Z-score (float)
    """
    mean = price_data[-window:].mean()
    std = price_data[-window:].std()
    z_score = (price_data[-1] - mean) / std
    
    return z_score

In [ ]:
dip_buying_strategy (data)

In [ ]:
####Momentus trading strategy###

import pandas as pd
import numpy as np

def momentum_trading_algorithm(data, window=14):
    # Calculate the momentum indicator
    momentum = data["Close"].diff(window)

    # Generate the trading signal
    signal = np.where(momentum > 0, 1, 0)

    # Create a DataFrame to store the results
    signal_df = pd.DataFrame(data=signal, columns=["Signal"], index=data.index)

    return signal_df

# Example usage
 
signal = momentum_trading_algorithm(data)
print(signal)

In [ ]:
def exponential_moving_average(data, window):
    alpha = 2 / (window + 1)
    ema = [data[0]]
    for i in range(1, len(data)):
        ema.append(alpha * data[i] + (1 - alpha) * ema[i - 1])
    return ema

In [ ]:
#regmine shift logic 1
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def regime_shift_trading_strategy(data, window=100, threshold=0.5):
    # Calculate the returns of the stock
    returns = data["Close"].pct_change().dropna()

    # Divide the returns into two regimes using a Random Forest classifier
    X = returns.values.reshape(-1, 1)
    y = (returns > returns.rolling(window).mean()).astype(int).values.ravel()

    clf.fit(X, y)

    regime = clf.predict_proba(X)[:, 11] #[:, 1]
    print (regime)
    
    # Generate the trading signal
    signal = np.where(regime > threshold, 1, -1) 
    print ('sginal', signal)
    # Create a DataFrame to store the results
    signal_df = pd.DataFrame(data=signal, columns=["Signal"], index=data.index)
   # print (signal_df)

    return signal_df

# Example usage
 
signal = regime_shift_trading_strategy(data)
print(signal)

In [ ]:
#regmine shift logic 2
import pandas as pd
import numpy as np

def regime_shift_strategy(data, window=20):
    # Calculate rolling average and standard deviation
#     roll_mean = data.rolling(window=20).mean()
#     roll_std = data.rolling(window=20).std()
    
    roll_mean = data['Close'].rolling(window).mean()
    roll_std = data['Close'].rolling(window).std()
    
    # Create a threshold by multiplying the standard deviation by a factor
    threshold = roll_mean + 2 * roll_std
 
    # Create a signal to buy (1) or sell (-1) based on the threshold
    signal = np.where(data > threshold, 1, -1)
    print (signal)   
    
    # Create a DataFrame to store the strategy results
    strategy = pd.DataFrame({'price': [data], 'signal': [signal]})
    return strategy

# Example usage
 
signal = regime_shift_strategy(data)
print(signal)

In [ ]:
#regmine shift logic 3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the stock price data into a pandas DataFrame
df = data
 
# Calculate some basic technical indicators as features
df["return"] = df["Close"].pct_change()
df["moving_avg"] = df["Close"].rolling(window=20).mean()
df["moving_std"] = df["Close"].rolling(window=20).std()
df["bollinger_upper"] = df["moving_avg"] + 2*df["moving_std"]
df["bollinger_lower"] = df["moving_avg"] - 2*df["moving_std"]
df["momentum"] = df["Close"] - df["Close"].shift(10)

# Create the target variable based on the regime shift (e.g., bullish or bearish market)
df["regime"] = np.where(df["return"] > 0, 1, 0)
 
df = df.dropna()
# Split the data into training and testing sets
features = ["moving_avg", "bollinger_upper", "bollinger_lower", "momentum"]
target = "regime"
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)
 
# extra start###############
# Separate the features and target
X = df.drop('regime', axis=1).dropna()
y = df['regime']
print ('y is', y)

# Replace negative values with 0.01
X[X < 0] = 0.01

# Train a RandomForestClassifier model on the input data
model = RandomForestClassifier()
model.fit(X, y)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2

# Select the best k features using chi2
selector = SelectKBest(chi2, k=5)
X = selector.fit_transform(X, y)

# Train an ExtraTreesClassifier model on the selected features
model = ExtraTreesClassifier()
model.fit(X, y)

# extra end###############
# Train the random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the classifier
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc*100))


# Plot the decision boundary
xx, yy = np.meshgrid(np.arange(X[:, 0].min() - 1, X[:, 0].max() + 1, 0.1),
                     np.arange(X[:, 1].min() - 1, X[:, 1].max() + 1, 0.1))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.RdBu)

# Plot the training and testing data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.RdBu, edgecolor='k')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.RdBu, edgecolor='k', alpha=0.6)

# Show the plot
plt.show()

#################################
# Plot the decision boundary
# Generate a sample dataset
# import matplotlib.pyplot as plt
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split

# X, y = make_classification(n_features=4, random_state=0)
# print (X)

# # Generate a grid of points for plotting the decision boundary
# xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))

# # Predict the class labels for the grid of points
# Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
# Z = Z.reshape(xx.shape)
# # Plot the training and testing data
# plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.RdBu, edgecolor='k')
# plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.RdBu, edgecolor='k', alpha=0.6)

# # Show the plot
# plt.show()

# Generate data with 100 features, 80 informative features, 10 redundant features, and 5 repeated features
# X, y = make_classification(n_features=100, n_informative=80, n_redundant=10, n_repeated=5, random_state=0)


##########extra start
# # Add 2 additional features to X to meet the requirement of the model
# X = np.hstack((X, np.zeros((100, 2))))
# print (X)
 
##########extra end

            # xx, yy = np.meshgrid(np.arange(X[:, 0].min() - 1, X[:, 0].max() + 1, 0.1),
            #                      np.arange(X[:, 1].min() - 1, X[:, 1].max() + 1, 0.1))

            # Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
            # plt.contourf(xx, yy, Z, cmap=plt.cm.RdBu)

            # # Plot the training and testing data
            # plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.RdBu, edgecolor='k')
            # plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.RdBu, edgecolor='k', alpha=0.6)

            # # Show the plot
            # plt.show()

#################################

# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# import numpy as np
 

# # Plot the training data
# plt.scatter(X_train , y_train, color='blue', marker='o', label='Training Data')

# # Plot the testing data
# # plt.scatter(X_test[:, 0], y_test, color='red', marker='x', label='Testing Data')
# plt.scatter(X_test["moving_avg"], y_test, color='red', marker='x', label='Testing Data')

# # Add a legend
# plt.legend(loc='best')

# # Show the plot
# plt.show()
